In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from itertools import combinations

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV

In [5]:
df = pd.read_excel("AIIMSData_SingleEntry.xlsx", sheet_name='Single entry per patient (2)')

In [6]:
# df2 = pd.read_excel('BTP_curr.xlsx', sheet_name='Sheet2')
# df2 = pd.read_excel("singleEntry.xlsx", sheet_name='Single entry per patient (2)')

In [7]:
# df = pd.concat([df1, df2], ignore_index=True)

In [8]:
# value_to_replace = ' '
# replacement_value = '35'
# df['lymphocytes'] = df['lymphocytes'].replace(value_to_replace, replacement_value)


In [9]:
# df['lymphocytes'] = df['lymphocytes'].astype(float)

In [10]:
# df['lymphocytes'].fillna(df['lymphocytes'].mean(), inplace=True)

In [11]:
df.fillna(0, inplace=True)

In [12]:
print(df.columns)

Index(['S/L NO', 'FILE NO', 'NAME', 'CRN', 'Age', 'Gender (1=Male/2=Female)',
       'Place of habitation(1=Urban,2=Rural)',
       'Annual income (1=BPL/2=Non BPL)\n',
       'Marital status (1=Married/0=Unmarried)',
       'Smoking status (1=smoker/0=non smoker)',
       'alcohol(1=non alchoholic 0=non alchoholic)',
       'Tobacco chewing status (1=yes/0=no)', 'Comorbidiites (1=Yes/0=No)',
       'If yes specify (1=T2DM/2=HTN/3=CAD/4=COPD/5=Multiple 6.=HEPATI 7.DM',
       'ECOG PS (1=0,1,2 2=3,4)', 'Body Mass Index (in Kg/m2)',
       'BMI (1=Normal/2=underweight/3=overweight/Obese)',
       'Bipedal edema (1=Yes/0=No)', 'Site of primary cancer', 'STAGE',
       'CHEMOTHERAY PROTOCOL (1.single agent\n2.doublet\n3.triplet)\n4.MULTIAGENT',
       ' cycle number  (1=1st cycle\n2=cycle 2 & beyond)\n',
       'Dosing of chemotherapy (1=Standard/2=Compromised)',
       'Use of prophylactic growth factors (1=Yes/0=No)',
       'Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 )',
       'WBC 

In [13]:
label_distribution = df['Severe Hematologic toxicity'].value_counts()
print(label_distribution)

Severe Hematologic toxicity
0    329
1    103
Name: count, dtype: int64


In [14]:
feature_All = df[['Age', 'Gender (1=Male/2=Female)',
       'Place of habitation(1=Urban,2=Rural)',
       'Annual income (1=BPL/2=Non BPL)\n',
       'Smoking status (1=smoker/0=non smoker)',
       'alcohol(1=non alchoholic 0=non alchoholic)',
       'Tobacco chewing status (1=yes/0=no)', 'Comorbidiites (1=Yes/0=No)',
       'ECOG PS (1=0,1,2 2=3,4)','BMI (1=Normal/2=underweight/3=overweight/Obese)',
       'Bipedal edema (1=Yes/0=No)', 'Site of primary cancer', 'STAGE',
       'CHEMOTHERAY PROTOCOL (1.single agent\n2.doublet\n3.triplet)\n4.MULTIAGENT',
       ' cycle number  (1=1st cycle\n2=cycle 2 & beyond)\n',
       'Dosing of chemotherapy (1=Standard/2=Compromised)',
       'Use of prophylactic growth factors (1=Yes/0=No)',
       'Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 )',
       'WBC   (1=<4000    2=4000-11000   3=>11000)2',
       'Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400)',
       'Absolute neutrophil count (1=<1600  2= 1600-8800  3=>8800)',
       'Neutrophil to Lymphocyte ratio(1=<2 2=2 3=>2)',
       'Total platelet count(1=<150000 2=150000-450000 3=>450000)',
       'Serum albumin (g/dl)    (1=<3.5    2=3.5-5.2     3=>5.2)',
       'Serum creatinine   (1=<0.7  2=0.7-1.3    3=>1.3)',
       'eosinophils  (1=<1   2=1-6   3=>6)', 'basophils  (1=0-2   2=>2)',
       'monocytes  (1=<2  2=2-10   3=>10)'
]]

In [15]:
feature_All.dtypes

Age                                                                          float64
Gender (1=Male/2=Female)                                                     float64
Place of habitation(1=Urban,2=Rural)                                         float64
Annual income (1=BPL/2=Non BPL)\n                                            float64
Smoking status (1=smoker/0=non smoker)                                       float64
alcohol(1=non alchoholic 0=non alchoholic)                                   float64
Tobacco chewing status (1=yes/0=no)                                          float64
Comorbidiites (1=Yes/0=No)                                                   float64
ECOG PS (1=0,1,2 2=3,4)                                                      float64
BMI (1=Normal/2=underweight/3=overweight/Obese)                              float64
Bipedal edema (1=Yes/0=No)                                                   float64
Site of primary cancer                                           

In [16]:
target = df['Severe Hematologic toxicity']

In [17]:
X_train1, X_test1, y_train, y_test = train_test_split(feature_All, target, test_size=0.2, random_state=42)

In [18]:
# Initialize MinMaxScaler
# standardising and normalisation
scaler = MinMaxScaler()

# Fit the scaler to the training data and transform it
X_train = scaler.fit_transform(X_train1)

# Transform the test data using the same scaler
X_test = scaler.transform(X_test1)

In [19]:
print(y_train.dtype)

int64


## RANDOM FOREST

In [20]:
clf = RandomForestClassifier(n_estimators = 20, random_state=42) # 20 decision trees ensemble model- random forest
# Training the classifier
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20, random_state=42)

In [21]:
# Get feature importances
feature_importances = clf.feature_importances_

# Get the indices of features sorted by their importance
indices = feature_importances.argsort()[::1]

feature_names = ['Age', 'Gender (1=Male/2=Female)','Place of habitation(1=Urban,2=Rural)','Annual income (1=BPL/2=Non BPL)\n','Smoking status (1=smoker/0=non smoker)',
       'alcohol(1=non alchoholic 0=non alchoholic)','Tobacco chewing status (1=yes/0=no)', 'Comorbidiites (1=Yes/0=No)',
       'ECOG PS (1=0,1,2 2=3,4)','BMI (1=Normal/2=underweight/3=overweight/Obese)','Bipedal edema (1=Yes/0=No)', 'Site of primary cancer', 'STAGE',
       'CHEMOTHERAY PROTOCOL (1.single agent\n2.doublet\n3.triplet)\n4.MULTIAGENT',' cycle number  (1=1st cycle\n2=cycle 2 & beyond)\n',
       'Dosing of chemotherapy (1=Standard/2=Compromised)','Use of prophylactic growth factors (1=Yes/0=No)',
       'Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 )', 'WBC   (1=<4000    2=4000-11000   3=>11000)2',
       'Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400)','Absolute neutrophil count (1=<1600  2= 1600-8800  3=>8800)',
       'Neutrophil to Lymphocyte ratio(1=<2 2=2 3=>2)','Total platelet count(1=<150000 2=150000-450000 3=>450000)','Serum albumin (g/dl)    (1=<3.5    2=3.5-5.2     3=>5.2)',
       'Serum creatinine   (1=<0.7  2=0.7-1.3    3=>1.3)','eosinophils  (1=<1   2=1-6   3=>6)', 'basophils  (1=0-2   2=>2)','monocytes  (1=<2  2=2-10   3=>10)']

# Print feature ranking
print("Feature ranking:")
for f in range(len(feature_names)):
    print("%d. Feature: %s (%f)" % (f + 1, feature_names[indices[f]], feature_importances[indices[f]]))

Feature ranking:
1. Feature: alcohol(1=non alchoholic 0=non alchoholic) (0.001195)
2. Feature: Site of primary cancer (0.003426)
3. Feature: Tobacco chewing status (1=yes/0=no) (0.004512)
4. Feature: Bipedal edema (1=Yes/0=No) (0.006934)
5. Feature: Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 ) (0.009868)
6. Feature: Smoking status (1=smoker/0=non smoker) (0.011027)
7. Feature: monocytes  (1=<2  2=2-10   3=>10) (0.024814)
8. Feature: basophils  (1=0-2   2=>2) (0.026726)
9. Feature: Annual income (1=BPL/2=Non BPL)
 (0.027315)
10. Feature: Comorbidiites (1=Yes/0=No) (0.030072)
11. Feature: Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400) (0.030196)
12. Feature: Place of habitation(1=Urban,2=Rural) (0.031597)
13. Feature: Dosing of chemotherapy (1=Standard/2=Compromised) (0.032221)
14. Feature: eosinophils  (1=<1   2=1-6   3=>6) (0.033079)
15. Feature: ECOG PS (1=0,1,2 2=3,4) (0.034689)
16. Feature: Gender (1=Male/2=Female) (0.035378)
17. Feature: Total platelet count(1=<150000

In [22]:
# Get feature importances
feature_importances = clf.feature_importances_

# Get the indices of features sorted by their importance
indices = feature_importances.argsort()[::1]

feature_names = ['Age', 'Gender (1=Male/2=Female)','Place of habitation(1=Urban,2=Rural)','Annual income (1=BPL/2=Non BPL)\n','Smoking status (1=smoker/0=non smoker)',
       'alcohol(1=non alchoholic 0=non alchoholic)','Tobacco chewing status (1=yes/0=no)', 'Comorbidiites (1=Yes/0=No)',
       'ECOG PS (1=0,1,2 2=3,4)','BMI (1=Normal/2=underweight/3=overweight/Obese)','Bipedal edema (1=Yes/0=No)', 'Site of primary cancer', 'STAGE',
       'CHEMOTHERAY PROTOCOL (1.single agent\n2.doublet\n3.triplet)\n4.MULTIAGENT',' cycle number  (1=1st cycle\n2=cycle 2 & beyond)\n',
       'Dosing of chemotherapy (1=Standard/2=Compromised)','Use of prophylactic growth factors (1=Yes/0=No)',
       'Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 )', 'WBC   (1=<4000    2=4000-11000   3=>11000)2',
       'Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400)','Absolute neutrophil count (1=<1600  2= 1600-8800  3=>8800)',
       'Neutrophil to Lymphocyte ratio(1=<2 2=2 3=>2)','Total platelet count(1=<150000 2=150000-450000 3=>450000)','Serum albumin (g/dl)    (1=<3.5    2=3.5-5.2     3=>5.2)',
       'Serum creatinine   (1=<0.7  2=0.7-1.3    3=>1.3)','eosinophils  (1=<1   2=1-6   3=>6)', 'basophils  (1=0-2   2=>2)','monocytes  (1=<2  2=2-10   3=>10)']

# Print feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. Feature %s %s (%f)" % (f + 1, indices[f],feature_names[indices[f]], feature_importances[indices[f]]))

Feature ranking:
1. Feature 5 alcohol(1=non alchoholic 0=non alchoholic) (0.001195)
2. Feature 11 Site of primary cancer (0.003426)
3. Feature 6 Tobacco chewing status (1=yes/0=no) (0.004512)
4. Feature 10 Bipedal edema (1=Yes/0=No) (0.006934)
5. Feature 17 Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 ) (0.009868)
6. Feature 4 Smoking status (1=smoker/0=non smoker) (0.011027)
7. Feature 27 monocytes  (1=<2  2=2-10   3=>10) (0.024814)
8. Feature 26 basophils  (1=0-2   2=>2) (0.026726)
9. Feature 3 Annual income (1=BPL/2=Non BPL)
 (0.027315)
10. Feature 7 Comorbidiites (1=Yes/0=No) (0.030072)
11. Feature 19 Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400) (0.030196)
12. Feature 2 Place of habitation(1=Urban,2=Rural) (0.031597)
13. Feature 15 Dosing of chemotherapy (1=Standard/2=Compromised) (0.032221)
14. Feature 25 eosinophils  (1=<1   2=1-6   3=>6) (0.033079)
15. Feature 8 ECOG PS (1=0,1,2 2=3,4) (0.034689)
16. Feature 1 Gender (1=Male/2=Female) (0.035378)
17. Feature 22 Tota

In [23]:
y_pred_rf = clf.predict(X_test)

In [24]:
accuracy = accuracy_score(y_test, y_pred_rf)
report = classification_report(y_test,y_pred_rf)
print(f'Accuracy: {accuracy}\nClassification Report:\n{report}')

Accuracy: 0.7586206896551724
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.89      0.85        66
           1       0.50      0.33      0.40        21

    accuracy                           0.76        87
   macro avg       0.65      0.61      0.62        87
weighted avg       0.73      0.76      0.74        87



In [25]:
param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [5, 10, 15, 20, 25],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [4, 8, 16]
}

In [26]:
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(n_estimators=20, random_state=42),
             param_grid={'max_depth': [5, 10, 15, 20, 25],
                         'min_samples_leaf': [4, 8, 16],
                         'min_samples_split': [5, 10, 15],
                         'n_estimators': [50, 100, 200, 400]},
             scoring='accuracy')

In [27]:
best_params = grid_search.best_params_

In [28]:
best_params

{'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 50}

**FORWARD SELECTION**

In [29]:
# initializing lists to store accuracy, classification reports, and selected feature names
accuracy_scores = []
classification_reports = []
selected_feature_names_list = []

# incrementally increase the number of features
for i in range(1, len(feature_importances) + 1):
    # select the top i features
    selected_features = indices[:i]
    selected_feature_names = [feature_names[idx] for idx in selected_features]

    # select the corresponding columns from the training and testing data
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    # clone the model to avoid altering the original
    model = clone(clf)

    # fit the model
    model.fit(X_train_selected, y_train)

    #  predictions on the test set
    y_pred = model.predict(X_test_selected)

    accuracy = accuracy_score(y_test, y_pred)

    report = classification_report(y_test, y_pred)

    # store the accuracy score, classification report, and selected feature names
    accuracy_scores.append(accuracy)
    classification_reports.append(report)
    selected_feature_names_list.append(selected_feature_names)

# Print the accuracy scores, classification reports, and selected feature names for each feature set
for num_features, accuracy, report, selected_feature_names in zip(range(1, len(feature_importances) + 1), accuracy_scores, classification_reports, selected_feature_names_list):
    print(f"Number of features: {num_features}")
    print(f"Selected feature names: {selected_feature_names}")
    print(f"Accuracy: {accuracy}")
    print(f"Classification Report:\n{report}\n")

c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Number of features: 1
Selected feature names: ['alcohol(1=non alchoholic 0=non alchoholic)']
Accuracy: 0.7586206896551724
Classification Report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86        66
           1       0.00      0.00      0.00        21

    accuracy                           0.76        87
   macro avg       0.38      0.50      0.43        87
weighted avg       0.58      0.76      0.65        87


Number of features: 2
Selected feature names: ['alcohol(1=non alchoholic 0=non alchoholic)', 'Site of primary cancer']
Accuracy: 0.7586206896551724
Classification Report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86        66
           1       0.00      0.00      0.00        21

    accuracy                           0.76        87
   macro avg       0.38      0.50      0.43        87
weighted avg       0.58      0.76      0.65        87


Number of features: 3
Select

**BACKWARD SELECTION**

In [30]:
# Initialize lists to store accuracy, classification reports, and selected feature names
accuracy_scores = []
classification_reports = []
selected_feature_names_list = []

indices = feature_importances.argsort()[::-1]

# Start with all 24 features and iteratively remove the less important ones
for i in range(len(feature_importances), 0, -1):
    # Select the top i features
    selected_features = indices[:i]
    selected_feature_names = [feature_names[idx] for idx in selected_features]

    # Select the corresponding columns from the training and testing data
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    # Clone the model to avoid altering the original
    model = clone(clf)

    # Fit the model
    model.fit(X_train_selected, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test_selected)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Generate classification report
    report = classification_report(y_test, y_pred)

    # Store the accuracy score, classification report, and selected feature names
    accuracy_scores.append(accuracy)
    classification_reports.append(report)
    selected_feature_names_list.append(selected_feature_names)

# Print the accuracy scores, classification reports, and selected feature names for each feature set
for num_features, accuracy, report, selected_feature_names in zip(range(len(feature_importances), 0, -1), accuracy_scores, classification_reports, selected_feature_names_list):
    print(f"Number of features: {num_features}")
    print(f"Selected feature names: {selected_feature_names}")
    print(f"Accuracy: {accuracy}")
    print(f"Classification Report:\n{report}\n")

Number of features: 28
Selected feature names: ['STAGE', 'Serum albumin (g/dl)    (1=<3.5    2=3.5-5.2     3=>5.2)', 'CHEMOTHERAY PROTOCOL (1.single agent\n2.doublet\n3.triplet)\n4.MULTIAGENT', 'Serum creatinine   (1=<0.7  2=0.7-1.3    3=>1.3)', 'BMI (1=Normal/2=underweight/3=overweight/Obese)', 'WBC   (1=<4000    2=4000-11000   3=>11000)2', 'Neutrophil to Lymphocyte ratio(1=<2 2=2 3=>2)', 'Absolute neutrophil count (1=<1600  2= 1600-8800  3=>8800)', 'Use of prophylactic growth factors (1=Yes/0=No)', 'Age', ' cycle number  (1=1st cycle\n2=cycle 2 & beyond)\n', 'Total platelet count(1=<150000 2=150000-450000 3=>450000)', 'Gender (1=Male/2=Female)', 'ECOG PS (1=0,1,2 2=3,4)', 'eosinophils  (1=<1   2=1-6   3=>6)', 'Dosing of chemotherapy (1=Standard/2=Compromised)', 'Place of habitation(1=Urban,2=Rural)', 'Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400)', 'Comorbidiites (1=Yes/0=No)', 'Annual income (1=BPL/2=Non BPL)\n', 'basophils  (1=0-2   2=>2)', 'monocytes  (1=<2  2=2-10   

c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

## LOGISTIC REGRESSION

In [31]:
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train, y_train)
y_pred_lr = log_reg_model.predict(X_test)

In [32]:
accuracy = log_reg_model.score(X_test, y_test)

In [33]:
# scaling the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Fit logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)

feature_names = ['Age', 'Gender (1=Male/2=Female)','Place of habitation(1=Urban,2=Rural)','Annual income (1=BPL/2=Non BPL)\n','Smoking status (1=smoker/0=non smoker)',
       'alcohol(1=non alchoholic 0=non alchoholic)','Tobacco chewing status (1=yes/0=no)', 'Comorbidiites (1=Yes/0=No)',
       'ECOG PS (1=0,1,2 2=3,4)','BMI (1=Normal/2=underweight/3=overweight/Obese)','Bipedal edema (1=Yes/0=No)', 'Site of primary cancer', 'STAGE',
       'CHEMOTHERAY PROTOCOL (1.single agent\n2.doublet\n3.triplet)\n4.MULTIAGENT',' cycle number  (1=1st cycle\n2=cycle 2 & beyond)\n',
       'Dosing of chemotherapy (1=Standard/2=Compromised)','Use of prophylactic growth factors (1=Yes/0=No)',
       'Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 )', 'WBC   (1=<4000    2=4000-11000   3=>11000)2',
       'Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400)','Absolute neutrophil count (1=<1600  2= 1600-8800  3=>8800)',
       'Neutrophil to Lymphocyte ratio(1=<2 2=2 3=>2)','Total platelet count(1=<150000 2=150000-450000 3=>450000)','Serum albumin (g/dl)    (1=<3.5    2=3.5-5.2     3=>5.2)',
       'Serum creatinine   (1=<0.7  2=0.7-1.3    3=>1.3)','eosinophils  (1=<1   2=1-6   3=>6)', 'basophils  (1=0-2   2=>2)','monocytes  (1=<2  2=2-10   3=>10)']

# Calculating VIF(variance inflation factor) for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = feature_names  # usig the original feature names
vif_data["VIF"] = [variance_inflation_factor(X_train_scaled, i) for i in range(X_train_scaled.shape[1])]

# sorting by VIF values
vif_data_sorted = vif_data.sort_values(by='VIF', ascending=False)

print(vif_data_sorted)

                                              feature        VIF
11                             Site of primary cancer  40.878653
17     Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 )  18.908247
22  Total platelet count(1=<150000 2=150000-450000...  17.481510
18        WBC   (1=<4000    2=4000-11000   3=>11000)2  14.673158
20  Absolute neutrophil count (1=<1600  2= 1600-88...  14.484017
14   cycle number  (1=1st cycle\n2=cycle 2 & beyon...  13.616736
1                            Gender (1=Male/2=Female)  11.317415
2                Place of habitation(1=Urban,2=Rural)  10.802120
0                                                 Age   9.397236
15  Dosing of chemotherapy (1=Standard/2=Compromised)   9.303571
8                             ECOG PS (1=0,1,2 2=3,4)   8.008318
3                   Annual income (1=BPL/2=Non BPL)\n   7.721951
27                  monocytes  (1=<2  2=2-10   3=>10)   6.014085
25                 eosinophils  (1=<1   2=1-6   3=>6)   5.957683
13  CHEMOTHERAY PROTOCOL 

**BACKWARD SELECTION ON THE FEATURES**

In [34]:
# def permutation_importance(model, X, y, metric, random_state=42):
#     baseline_score = metric(y, model.predict(X))
#     scores = {}
#     n_features = X.shape[1]
#     for r in range(1, n_features + 1):  # Try all combinations of features from 1 to n_features
#         for feature_indices in combinations(range(n_features), r):
#             X_permuted = X.copy()
#             for feature_idx in feature_indices:
#                 permuted_idx = np.random.permutation(X.shape[0])
#                 X_permuted[:, feature_idx] = X_permuted[permuted_idx, feature_idx]
#             permuted_score = metric(y, model.predict(X_permuted))
#             feature_names_permuted = ', '.join([feature_names[i] for i in feature_indices])
#             scores[feature_names_permuted] = permuted_score
#     return scores

def permutation_importance(model, X, y, metric, random_state=42):
    baseline_score = metric(y, model.predict(X))
    scores = {}
    for i, feature_name in enumerate(feature_names):
        X_permuted = X.copy()
        permuted_idx = np.random.permutation(X.shape[0])
        X_permuted[:, i] = X_permuted[permuted_idx, i]
        permuted_score = metric(y, model.predict(X_permuted))
        scores[feature_name] = permuted_score
    return scores

In [35]:
# Train initial logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
initial_accuracy = accuracy_score(y_test, logreg.predict(X_test))
print("Initial Accuracy:", initial_accuracy)

# Start with all features
selected_features = list(feature_All.columns)

feature_importances = clf.feature_importances_

feature_importance_dict = dict(zip(X_train1.columns, abs(feature_importances)))

# Sort features by importance scores in ascending order
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1])

# Start with all features
selected_features = ['Age', 'Gender (1=Male/2=Female)','Place of habitation(1=Urban,2=Rural)','Annual income (1=BPL/2=Non BPL)\n','Smoking status (1=smoker/0=non smoker)',
       'alcohol(1=non alchoholic 0=non alchoholic)','Tobacco chewing status (1=yes/0=no)', 'Comorbidiites (1=Yes/0=No)',
       'ECOG PS (1=0,1,2 2=3,4)','BMI (1=Normal/2=underweight/3=overweight/Obese)','Bipedal edema (1=Yes/0=No)', 'Site of primary cancer', 'STAGE',
       'CHEMOTHERAY PROTOCOL (1.single agent\n2.doublet\n3.triplet)\n4.MULTIAGENT',' cycle number  (1=1st cycle\n2=cycle 2 & beyond)\n',
       'Dosing of chemotherapy (1=Standard/2=Compromised)','Use of prophylactic growth factors (1=Yes/0=No)',
       'Hemoglobin (g/dl) ( 1=<13    2=13-17   3=>17 )', 'WBC   (1=<4000    2=4000-11000   3=>11000)2',
       'Absolute Lymphocyte count  (1=<800   2=800-4400   3=>4400)','Absolute neutrophil count (1=<1600  2= 1600-8800  3=>8800)',
       'Neutrophil to Lymphocyte ratio(1=<2 2=2 3=>2)','Total platelet count(1=<150000 2=150000-450000 3=>450000)','Serum albumin (g/dl)    (1=<3.5    2=3.5-5.2     3=>5.2)',
       'Serum creatinine   (1=<0.7  2=0.7-1.3    3=>1.3)','eosinophils  (1=<1   2=1-6   3=>6)', 'basophils  (1=0-2   2=>2)','monocytes  (1=<2  2=2-10   3=>10)']


for removed_feature in sorted_features[:]:
    # Train logistic regression model with reduced features
    removed_feature_name = removed_feature
    if removed_feature_name in selected_features:
       selected_features.remove(removed_feature_name)
       logreg.fit(X_train1[selected_features], y_train)
       updated_accuracy = accuracy_score(y_test, logreg.predict(X_test1[selected_features]))
   
       print(f"Removed Feature {removed_feature}, Updated Accuracy: {updated_accuracy}")
   
       # If accuracy drops below a threshold, stop the process
       if updated_accuracy < initial_accuracy - 1:
           print("Accuracy dropped below threshold. Stopping backward selection.")
           break
       else:
           # Update initial accuracy for the next iteration
           initial_accuracy = updated_accuracy

# Train final model with selected features
# logreg.fit(X_train1[selected_features], y_train)
# final_accuracy = accuracy_score(y_test, logreg.predict(X_test[selected_features]))
# print("Final Accuracy with Selected Features:", final_accuracy)

Initial Accuracy: 0.8045977011494253


In [36]:
model = clone(logreg)

# Fit the model
model.fit(X_train, y_train)

# Calculate baseline accuracy using cross-validation
baseline_accuracy = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()

# Calculate permutation feature importance
importance_scores = permutation_importance(model, X_train, y_train, accuracy_score)

# Select important features (e.g., top N features)
n_features_to_select = 28
important_features = sorted(importance_scores, key=importance_scores.get, reverse=True)[:n_features_to_select]

# Retrain the model using only the important features
important_feature_indices = [feature_names.index(feature) for feature in important_features]
X_train_selected = X_train[:, important_feature_indices]
model.fit(X_train_selected, y_train)


LogisticRegression()

In [37]:
# Calculate permutation accuracy for each feature
permutation_accuracy_scores = permutation_importance(model, X_train, y_train, accuracy_score)

# Find the feature with the highest accuracy when permuted
highest_accuracy_feature = max(permutation_accuracy_scores, key=permutation_accuracy_scores.get)
highest_accuracy = permutation_accuracy_scores[highest_accuracy_feature]

print("Highest accuracy obtained by permuting feature:", highest_accuracy_feature)
print("Accuracy:", highest_accuracy)

Highest accuracy obtained by permuting feature: Comorbidiites (1=Yes/0=No)
Accuracy: 0.7594202898550725


In [38]:
# Calculate accuracy using cross-validation on the new permutation models
new_accuracy_scores = cross_val_score(model, X_train_selected, y_train, cv=5, scoring='accuracy')

# Mean accuracy of the new permutation models
mean_new_accuracy = new_accuracy_scores.mean()

print("Mean accuracy of the new permutation models:", mean_new_accuracy)

Mean accuracy of the new permutation models: 0.7565217391304347


In [39]:
report = classification_report(y_pred_lr,y_test)

print(f'Accuracy: {accuracy}\nClassification Report:\n{report}')

Accuracy: 0.8045977011494253
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.80      0.88        81
           1       0.24      0.83      0.37         6

    accuracy                           0.80        87
   macro avg       0.61      0.82      0.63        87
weighted avg       0.93      0.80      0.85        87



## KNN

In [40]:
k=3
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [41]:
y_pred_kn = knn_model.predict(X_test)

In [42]:
print("Accuracy:", accuracy_score(y_test, y_pred_kn))
print(classification_report(y_test, y_pred_kn))

Accuracy: 0.7931034482758621
              precision    recall  f1-score   support

           0       0.84      0.89      0.87        66
           1       0.59      0.48      0.53        21

    accuracy                           0.79        87
   macro avg       0.72      0.69      0.70        87
weighted avg       0.78      0.79      0.79        87



## SVM

In [43]:
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

In [44]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.77      1.00      0.87        66
           1       1.00      0.05      0.09        21

    accuracy                           0.77        87
   macro avg       0.88      0.52      0.48        87
weighted avg       0.82      0.77      0.68        87



In [45]:
accuracy = svm_model.score(X_test, y_test)

In [46]:
accuracy

0.7701149425287356

## GRADIENT BOOSTING

In [47]:
# Create and train the model
gbm = GradientBoostingClassifier()
gbm.fit(X_train, y_train)
y_pred_xgb = gbm.predict(X_test)
# Evaluate the model
accuracy = gbm.score(X_test, y_test)
print(accuracy)
print(classification_report(y_test, y_pred_xgb))

0.8160919540229885
              precision    recall  f1-score   support

           0       0.85      0.92      0.88        66
           1       0.67      0.48      0.56        21

    accuracy                           0.82        87
   macro avg       0.76      0.70      0.72        87
weighted avg       0.80      0.82      0.80        87



## (Multi-layer Perceptron)

In [48]:
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100, ),  # Size of hidden layers
    activation='relu',           # Activation function for hidden layers
    solver='adam',               # Optimization algorithm
    alpha=0.0001,                # L2 regularization parameter
    learning_rate='constant',    # Learning rate schedule
    learning_rate_init=0.001,    # Initial learning rate
    max_iter=1000,               # Maximum number of iterations
    tol=1e-4,                    # Tolerance for optimization
    early_stopping=False,        # Whether to use early stopping
    validation_fraction=0.1,     # Fraction of training data to use as validation set if early stopping is enabled
    beta_1=0.9,                  # Exponential decay rate for estimates of first moment vector in adam
    beta_2=0.999,                # Exponential decay rate for estimates of second moment vector in adam
    epsilon=1e-8,                # Value for numerical stability in adam
    n_iter_no_change=10,         # Maximum number of epochs to not meet tol improvement
    random_state=42              # Random seed for reproducibility
)

In [49]:
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_test)
# Evaluate the model
accuracy = mlp_model.score(X_test, y_test)
print(accuracy)
print(classification_report(y_test, y_pred_mlp))

0.7471264367816092
              precision    recall  f1-score   support

           0       0.81      0.86      0.84        66
           1       0.47      0.38      0.42        21

    accuracy                           0.75        87
   macro avg       0.64      0.62      0.63        87
weighted avg       0.73      0.75      0.74        87



c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


## Stacking Ensemble


In [50]:
from sklearn.ensemble import StackingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

In [51]:
base_models = [
    ('rf', RandomForestClassifier(max_depth=5, min_samples_leaf=8, min_samples_split=5, n_estimators=50, random_state=42)),
    ('log_reg', LogisticRegression()),
    ('knn', KNeighborsClassifier(n_neighbors=3)),
    ('svm', SVC(probability=True)),  # Ensure SVC has probability=True for stacking
    ('gbm', GradientBoostingClassifier()),
    ('mlp', MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000, random_state=42))
]

In [52]:
# Define the meta-model
meta_model = LogisticRegression()

# Create the StackingClassifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method='predict_proba'  # Use predict_proba for classification
)

In [53]:
# Train the stacking model
stacking_clf.fit(X_train, y_train)

# Evaluate the stacking model
y_pred_stack = stacking_clf.predict(X_test)

c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\likhi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the 

In [54]:
accuracy_stack = accuracy_score(y_test, y_pred_stack)
print(f'Stacking Classifier Accuracy: {accuracy_stack}')
print(classification_report(y_test, y_pred_stack))

Stacking Classifier Accuracy: 0.8275862068965517
              precision    recall  f1-score   support

           0       0.84      0.95      0.89        66
           1       0.75      0.43      0.55        21

    accuracy                           0.83        87
   macro avg       0.79      0.69      0.72        87
weighted avg       0.82      0.83      0.81        87



In [55]:
pickle.dump(clf, open("singleHematoSevere.pkl", 'wb'))